# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9fa0421cd0>
├── 'a' --> tensor([[-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378]])
└── 'x' --> <FastTreeValue 0x7f9fa0421760>
    └── 'c' --> tensor([[ 0.4494, -0.3140, -0.2337,  0.3891],
                        [-1.7179,  0.2020,  1.1379,  0.4808],
                        [-0.0093, -2.0655,  1.6609, -0.5265]])

In [4]:
t.a

tensor([[-1.3318,  0.9827, -1.0321],
        [-0.0648,  0.0106,  0.7378]])

In [5]:
%timeit t.a

62.6 ns ± 0.172 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9fa0421cd0>
├── 'a' --> tensor([[ 0.8381,  1.9364,  1.0099],
│                   [-0.2453, -0.0662, -0.8854]])
└── 'x' --> <FastTreeValue 0x7f9fa0421760>
    └── 'c' --> tensor([[ 0.4494, -0.3140, -0.2337,  0.3891],
                        [-1.7179,  0.2020,  1.1379,  0.4808],
                        [-0.0093, -2.0655,  1.6609, -0.5265]])

In [7]:
%timeit t.a = new_value

66.3 ns ± 0.126 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3318,  0.9827, -1.0321],
               [-0.0648,  0.0106,  0.7378]]),
    x: Batch(
           c: tensor([[ 0.4494, -0.3140, -0.2337,  0.3891],
                      [-1.7179,  0.2020,  1.1379,  0.4808],
                      [-0.0093, -2.0655,  1.6609, -0.5265]]),
       ),
)

In [10]:
b.a

tensor([[-1.3318,  0.9827, -1.0321],
        [-0.0648,  0.0106,  0.7378]])

In [11]:
%timeit b.a

60.3 ns ± 0.0343 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2537,  1.0353,  0.6541],
               [-0.8020, -0.6218,  0.2179]]),
    x: Batch(
           c: tensor([[ 0.4494, -0.3140, -0.2337,  0.3891],
                      [-1.7179,  0.2020,  1.1379,  0.4808],
                      [-0.0093, -2.0655,  1.6609, -0.5265]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.0574 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

827 ns ± 0.929 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 15 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9fa0418910>
├── 'a' --> tensor([[[-1.3318,  0.9827, -1.0321],
│                    [-0.0648,  0.0106,  0.7378]],
│           
│                   [[-1.3318,  0.9827, -1.0321],
│                    [-0.0648,  0.0106,  0.7378]],
│           
│                   [[-1.3318,  0.9827, -1.0321],
│                    [-0.0648,  0.0106,  0.7378]],
│           
│                   [[-1.3318,  0.9827, -1.0321],
│                    [-0.0648,  0.0106,  0.7378]],
│           
│                   [[-1.3318,  0.9827, -1.0321],
│                    [-0.0648,  0.0106,  0.7378]],
│           
│                   [[-1.3318,  0.9827, -1.0321],
│                    [-0.0648,  0.0106,  0.7378]],
│           
│                   [[-1.3318,  0.9827, -1.0321],
│                    [-0.0648,  0.0106,  0.7378]],
│           
│                   [[-1.3318,  0.9827, -1.0321],
│                    [-0.0648,  0.0106,  0.7378]]])
└── 'x' --> <FastTreeValue 0x7f9ee9ccccd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 37.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9ee96b3b50>
├── 'a' --> tensor([[-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378],
│                   [-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378],
│                   [-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378],
│                   [-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378],
│                   [-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378],
│                   [-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378],
│                   [-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378],
│                   [-1.3318,  0.9827, -1.0321],
│                   [-0.0648,  0.0106,  0.7378]])
└── 'x' --> <FastTreeValue 0x7f9ee96b3910>
    └── 'c' --> tensor([[ 0.4494, -0.3140, -0.2337,  0.3891],
                        [-1.7179,  0.2020,  1.1379,  0.4808],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 50.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 43.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.4494, -0.3140, -0.2337,  0.3891],
                       [-1.7179,  0.2020,  1.1379,  0.4808],
                       [-0.0093, -2.0655,  1.6609, -0.5265]],
              
                      [[ 0.4494, -0.3140, -0.2337,  0.3891],
                       [-1.7179,  0.2020,  1.1379,  0.4808],
                       [-0.0093, -2.0655,  1.6609, -0.5265]],
              
                      [[ 0.4494, -0.3140, -0.2337,  0.3891],
                       [-1.7179,  0.2020,  1.1379,  0.4808],
                       [-0.0093, -2.0655,  1.6609, -0.5265]],
              
                      [[ 0.4494, -0.3140, -0.2337,  0.3891],
                       [-1.7179,  0.2020,  1.1379,  0.4808],
                       [-0.0093, -2.0655,  1.6609, -0.5265]],
              
                      [[ 0.4494, -0.3140, -0.2337,  0.3891],
                       [-1.7179,  0.2020,  1.1379,  0.4808],
                       [-0.0093, -2.0655,  1.6609, -0.5265]],

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 500 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.4494, -0.3140, -0.2337,  0.3891],
                      [-1.7179,  0.2020,  1.1379,  0.4808],
                      [-0.0093, -2.0655,  1.6609, -0.5265],
                      [ 0.4494, -0.3140, -0.2337,  0.3891],
                      [-1.7179,  0.2020,  1.1379,  0.4808],
                      [-0.0093, -2.0655,  1.6609, -0.5265],
                      [ 0.4494, -0.3140, -0.2337,  0.3891],
                      [-1.7179,  0.2020,  1.1379,  0.4808],
                      [-0.0093, -2.0655,  1.6609, -0.5265],
                      [ 0.4494, -0.3140, -0.2337,  0.3891],
                      [-1.7179,  0.2020,  1.1379,  0.4808],
                      [-0.0093, -2.0655,  1.6609, -0.5265],
                      [ 0.4494, -0.3140, -0.2337,  0.3891],
                      [-1.7179,  0.2020,  1.1379,  0.4808],
                      [-0.0093, -2.0655,  1.6609, -0.5265],
                      [ 0.4494, -0.3140, -0.2337,  0.3891],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 224 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 2.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
